In [1]:
from funcx.sdk.client import FuncXClient
import time


fxc = FuncXClient()
fxc.throttling_enabled = False
print(fxc.base_url, fxc.throttling_enabled)

https://funcx.org/api/v1 False


# Functions to work with endpoints

In [2]:
endpoints = ['709118de-1103-463f-8425-281eb93b55ff',   #Theta covid19
             '67e95158-8bda-4b1f-a0ef-31a1626eba00',   #Ryan ep
            ]

In [3]:
def get_endpoints_status(endpoints):
    endpoint_status = {}
    idle_workers = {}
    for ep in endpoints:
        endpoint_status[ep] = fxc.get_endpoint_status(ep)[0]
        idle_workers[ep] = endpoint_status[ep]['idle_workers']
    return endpoint_status, idle_workers

# fx functions

In [4]:
def warm_endpoint():
    """A funcX function to warm the endpoint 
    and create workers"""
    pass

In [5]:
def funcx_runner(index, filename, batchsize, index_start, index_end, workdir=None, out_file=None):
    """funcX function to perform work"""
    from candle_apps.candle_node_local import funcx_node_local
    return funcx_node_local(filename, index, batchsize, index_start, index_end,
                            workdir=workdir, out_file=out_file)

In [6]:
func_uuid2 = fxc.register_function(funcx_runner,
                                   description="A funcx function for covid19")

print(func_uuid2)

cf1b53e1-c2e6-4b0e-b9f6-4604509fb3b3


# Gerate workloads

In [7]:
def generate_batch(filename, start=0, batchsize=10, max_batches=10):
    counter = 0
    if max_batches == 0:
        max_batches = 999999999

    x = 'Hello'
    batch_index = []
    with open(filename) as current:
        batch_index.append(current.tell())
        counter += 1

        while x and counter < max_batches:
            counter += 1
            for i in range(batchsize):
                x = current.readline()

            batch_index.append(current.tell())
        return batch_index

In [8]:
func_uuid1 = fxc.register_function(generate_batch,
                                   description="A funcx function for generating batch")

print(func_uuid1)

ae6bf2b4-f62d-456f-b5e1-7fbceedf8948


In [9]:
res1 = fxc.run(filename='/home/rchard/src/covid19/ScreenPilot/ena+db.can',
               endpoint_id='67e95158-8bda-4b1f-a0ef-31a1626eba00',
               function_id=func_uuid1)

print(res1)

f388d941-72db-47b6-ab1d-9d6587c44f57


In [10]:
batch_index = fxc.get_result(res1)
print(batch_index)

[0, 2321, 3153, 3480, 4006, 4429, 4950, 5357, 5994, 6402]


# Distribute work

Define configs for the two eps

In [11]:
configs = {'709118de-1103-463f-8425-281eb93b55ff': {'filename': '/home/zzli/candle/ScreenPilot/ena+db.can',
                                                    'workdir': '/tmp/zzli/',
                                                    'out_path': '/home/zzli/candle/ScreenPilot/'},
           '67e95158-8bda-4b1f-a0ef-31a1626eba00': {'filename': '/home/rchard/src/covid19/ScreenPilot/ena+db.can',
                                                    'workdir': '/tmp/rchard/',
                                                    'out_path': '/home/rchard/src/covid19/ScreenPilot/'}
          }

Submit workloads

In [12]:
def submit_job(endpoint_uuid, idle_workers, idx):
    exec_config = configs[endpoint_uuid]
    res = fxc.run(filename=exec_config['filename'],
              index=idx,
              batchsize=10,
              index_start=0,
              index_end=1000,
              workdir=exec_config['workdir'],
              out_file=f'{exec_config["out_path"]}data-{idx}',
              endpoint_id=endpoint_uuid,
              function_id=func_uuid2)
    return res

In [13]:
def do_work(batch_index):
    task_ids = []
    while len(batch_index) > 0:
        stats, idle = get_endpoints_status(endpoints)
        for ep, idle_workers in idle.items():
            for x in range(idle_workers):
                try:
                    idx = batch_index.pop()
                    print(f'submitting {idx} to {ep}')
                    task_ids.append(submit_job(ep, idle, idx))
                except IndexError as e:
                    print('Finished!')
                    return task_ids
        time.sleep(5)
    return task_ids

In [14]:
task_ids = do_work(batch_index)

submitting 6402 to 709118de-1103-463f-8425-281eb93b55ff
submitting 5994 to 67e95158-8bda-4b1f-a0ef-31a1626eba00
submitting 5357 to 67e95158-8bda-4b1f-a0ef-31a1626eba00
submitting 4950 to 709118de-1103-463f-8425-281eb93b55ff
submitting 4429 to 67e95158-8bda-4b1f-a0ef-31a1626eba00
submitting 4006 to 67e95158-8bda-4b1f-a0ef-31a1626eba00
submitting 3480 to 709118de-1103-463f-8425-281eb93b55ff
submitting 3153 to 67e95158-8bda-4b1f-a0ef-31a1626eba00
submitting 2321 to 67e95158-8bda-4b1f-a0ef-31a1626eba00
submitting 0 to 709118de-1103-463f-8425-281eb93b55ff
Finished!


In [15]:
print(task_ids)

['d8c301d5-4ca3-41bd-a2b0-9a802d137d3a', 'ae89b6b3-edb3-49ff-a9de-9a6fc100b9b5', '3d3024b4-5632-4498-9658-19b9d63c05bf', 'd8eee917-12f6-4533-b21c-08651b6edbf8', 'c919e5af-41a0-4301-951f-f79d57b9848c', 'bd0b8c4e-e115-4c09-b59f-80be633bfccc', '6febd111-a6bd-4342-a1d3-aa6cad75233f', '7cbd9fe5-8ec8-4d68-89f2-05955878866d', 'a6af434a-c758-4cfd-acf5-9aaf177c171c', '105f71b3-6f58-47a1-86a5-8dc07cadefd1']


In [16]:
x = fxc.get_batch_status(task_ids)
print(x)
try:
    print(x[res[0]]['exception'].reraise())
except:
    pass

{'d8c301d5-4ca3-41bd-a2b0-9a802d137d3a': {'pending': 'False', 'result': '/home/zzli/candle/ScreenPilot/data-6402'}, 'ae89b6b3-edb3-49ff-a9de-9a6fc100b9b5': {'pending': 'False', 'result': '/home/rchard/src/covid19/ScreenPilot/data-5994'}, '3d3024b4-5632-4498-9658-19b9d63c05bf': {'pending': 'False', 'result': '/home/rchard/src/covid19/ScreenPilot/data-5357'}, 'd8eee917-12f6-4533-b21c-08651b6edbf8': {'pending': 'False', 'result': '/home/zzli/candle/ScreenPilot/data-4950'}, 'c919e5af-41a0-4301-951f-f79d57b9848c': {'pending': 'False', 'result': '/home/rchard/src/covid19/ScreenPilot/data-4429'}, 'bd0b8c4e-e115-4c09-b59f-80be633bfccc': {'pending': 'False', 'result': '/home/rchard/src/covid19/ScreenPilot/data-4006'}, '6febd111-a6bd-4342-a1d3-aa6cad75233f': {'pending': 'False', 'result': '/home/zzli/candle/ScreenPilot/data-3480'}, '7cbd9fe5-8ec8-4d68-89f2-05955878866d': {'pending': 'False', 'result': '/home/rchard/src/covid19/ScreenPilot/data-3153'}, 'a6af434a-c758-4cfd-acf5-9aaf177c171c': {'pe